In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "../Resources/schools_complete.csv"
student_data_to_load = "../Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
# check length of list of unique school names to get # of schools
school_count = len(school_data_complete["school_name"].unique())

# check length of list of unique student names to get # of students
student_count = len(school_data_complete["student_name"])

# sum budget data from the school data dataframe
total_budget = school_data["budget"].sum()

# average student's math scores
avg_math = school_data_complete["math_score"].mean()

# average student's reading scores
avg_read = school_data_complete["reading_score"].mean()

# calc # of students passing by getting length of list of passing students
m_passing_students = len(school_data_complete.loc[school_data_complete["math_score"]>=70,:])
r_passing_students = len(school_data_complete.loc[school_data_complete["reading_score"]>=70,:])
passing_students = len(school_data_complete.loc[(school_data_complete["reading_score"]>=70)&(school_data_complete["math_score"]>=70),:])

In [4]:
# calc % passing math
per_pass_math = (m_passing_students / student_count)*100
per_pass_math

74.9808526933878

In [5]:
# calc % passing reading
per_pass_read = (r_passing_students / student_count)*100
per_pass_read

85.80546336482001

In [6]:
# calc % passing both
per_pass = (passing_students / student_count)*100
per_pass

65.17232575950983

In [7]:
# create dataframe using the above calculated values
dist_summary = pd.DataFrame({"Total Schools":[school_count],
                            "Total Students":student_count,
                            "Total Budget":total_budget,
                            "Average Math Score":avg_math,
                            "Average Reading Score":avg_read,
                            "% Passing Math":per_pass_math,
                            "% Passing Reading":per_pass_read,
                            "% Overall Passing":per_pass_math})


In [8]:
dist_summary["Total Budget"] = dist_summary["Total Budget"].map("${:,.2f}".format)
dist_summary["Total Students"] = dist_summary["Total Students"].map("{:,}".format)

In [9]:

dist_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,74.980853


In [10]:
# group school data to get AGGREGATED values from the merged dataframe
# remember that grouped_school is NOT a dataframe now but a groupby OBJECT
grouped_school = school_data_complete.groupby(['school_name','type'])


In [11]:
# get list of students passing math, reading, and both
passing_math = school_data_complete.loc[school_data_complete["math_score"]>=70,:]
passing_reading = school_data_complete.loc[school_data_complete["reading_score"]>=70,:]
passing_both = school_data_complete.loc[(school_data_complete["math_score"]>=70)&(school_data_complete["reading_score"]>=70),:]

In [12]:
# group all of the above
group_math = passing_math.groupby("school_name").count()["student_name"]
group_reading = passing_reading.groupby("school_name").count()["student_name"]
group_both = passing_both.groupby("school_name").count()["student_name"]

In [13]:
# calculate the desired fields using aggregate methods on the groupby object
school_students = grouped_school['student_name'].count()
type_school = grouped_school['type']
total_budget = grouped_school['budget'].mean()
per_student_budget = total_budget / school_students
avg_math_score = grouped_school['math_score'].mean()
avg_reading_score = grouped_school['reading_score'].mean()


In [14]:
sc_per_pass_math = (group_math / school_students)*100
sc_per_pass_reading = (group_reading / school_students)*100
sc_per_pass_both = (group_both / school_students)*100

In [15]:
school_summary = pd.DataFrame({'Total Students':school_students,
                               'Total School Budget':total_budget,
                               'Per Student Budget':per_student_budget,
                               'Average Math Score':avg_math_score,
                               'Average Reading Score':avg_reading_score,
                               'Percentage Passing Math':sc_per_pass_math,
                               'Percentage Passing Reading':sc_per_pass_reading,
                               'Percentage Passing Overall':sc_per_pass_both,
})


In [16]:
bins = [0,584.99,629.99,644.99,680]
group_names = ["<$585","$585-630","$630-645","$645-680"]
school_summary["Spending Ranges (Per Student)"]=pd.cut(school_summary["Per Student Budget"],bins,
                                                       labels = group_names,include_lowest = True)

In [17]:
bins2 = [0,999,1999,5000]
group_names2 = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]
school_summary["School Size"]=pd.cut(school_summary["Total Students"],bins2,
                                                       labels = group_names2,include_lowest = True)

In [18]:
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Overall,Spending Ranges (Per Student),School Size
school_name,type,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,<$585,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680,Large (2000-5000)


In [19]:
top_schools = school_summary.sort_values("Percentage Passing Overall", ascending= False)
top_schools.head(5)

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Overall,Spending Ranges (Per Student),School Size
school_name,type,,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585,Medium (1000-2000)
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012,$630-645,Medium (1000-2000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630,Medium (1000-2000)
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567,<$585,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,$585-630,Small (<1000)


In [20]:
bottom_schools = school_summary.sort_values("Percentage Passing Overall")
bottom_schools.head(5)

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Overall,Spending Ranges (Per Student),School Size
school_name,type,,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247,$630-645,Large (2000-5000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645,Large (2000-5000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680,Large (2000-5000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680,Large (2000-5000)


In [21]:
ninth = school_data_complete.loc[school_data_complete["grade"]=="9th",:]
tenth = school_data_complete.loc[school_data_complete["grade"]=="10th",:]
eleventh = school_data_complete.loc[school_data_complete["grade"]=="11th",:]
twelvth = school_data_complete.loc[school_data_complete["grade"]=="12th",:]

In [22]:

group_9_math = ninth.groupby("school_name").mean()["math_score"]

group_10_math = tenth.groupby("school_name").mean()["math_score"]

group_11_math = eleventh.groupby("school_name").mean()["math_score"]

group_12_math = twelvth.groupby("school_name").mean()["math_score"]

In [23]:
grade_math_summary = pd.DataFrame({"9th":group_9_math,
                                   "10th":group_10_math,
                                   "11th":group_11_math,
                                   "12th":group_12_math
})
grade_math_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [24]:

group_9_read = ninth.groupby("school_name").mean()["reading_score"]

group_10_read = tenth.groupby("school_name").mean()["reading_score"]

group_11_read = eleventh.groupby("school_name").mean()["reading_score"]

group_12_read = twelvth.groupby("school_name").mean()["reading_score"]

In [25]:
grade_read_summary = pd.DataFrame({"9th":group_9_read,
                                   "10th":group_10_read,
                                   "11th":group_11_read,
                                   "12th":group_12_read
})
grade_read_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [26]:
grouped_spend = school_summary.groupby("Spending Ranges (Per Student)")
grouped_spend.mean()[["Average Math Score","Average Reading Score","Percentage Passing Math","Percentage Passing Reading",
                    "Percentage Passing Overall"]]

,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


In [27]:
grouped_size = school_summary.groupby("School Size")
grouped_size.mean()[["Average Math Score","Average Reading Score","Percentage Passing Math","Percentage Passing Reading",
                    "Percentage Passing Overall"]]

,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Overall
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [28]:
grouped_type = school_summary.groupby("type")
grouped_type.mean()[["Average Math Score","Average Reading Score","Percentage Passing Math","Percentage Passing Reading",
                    "Percentage Passing Overall"]]

,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Overall
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
